# **ANÁLISIS EXPLORATORIO DE DATOS PARTE 1**

1. Funciones de resumen

2. Uso de función map

3. Uso de función groupby

4. Uso de multi-índices

5. Uso de la función sort

<img src="../imgs/foto-dia-03.jpg" width="500px" height="300px">

En la anterior clase aprendimos a seleccionar datos relevantes de un DataFrame o Serie. Extraer los datos correctos de nuestra representación de datos es fundamental para realizar el trabajo, como vimos en los ejercicios.

Sin embargo, los datos no siempre están en el formato que deseamos. A veces tenemos que hacer algo más de trabajo nosotros mismos para reformatearlos para la tarea que tenemos entre manos.

In [ ]:
import pandas as pd

compras = pd.read_csv("datasets/compras_centro_comercial.csv")

compras

### **1. Funciones de resumen**

Pandas proporciona muchas "funciones de resumen" simples (no es un nombre oficial) que reestructuran los datos de alguna manera útil. Por ejemplo, el método describe():

In [ ]:
compras["Annual Income"].describe()

Este método genera un resumen de estadísticas de los atributos de la columna dada. Su salida cambia en función del tipo de datos de entrada. La salida anterior sólo tiene sentido para datos numéricos; para datos de string esto es lo que obtenemos:

In [ ]:
compras["Gender"].describe()

Si deseas obtener alguna estadística de resumen simple sobre una columna en un DataFrame o una Serie, normalmente hay una función de pandas útil que lo hace posible.

Por ejemplo, para ver la media de los ingresos anuales de los compradores, podemos usar la función mean():

In [ ]:
compras["Annual Income"].mean()

Para ver una lista de valores únicos, podemos utilizar el método unique():

In [ ]:
compras["Gender"].unique()

Para ver una lista de valores únicos y la frecuencia con la que aparecen en el conjunto de datos, podemos utilizar el método value_counts():

In [ ]:
compras["Age"].value_counts()

### **2. Uso de función map**

Un mapa es un término, tomado de las matemáticas, para una función que toma un conjunto de valores y los "mapea" a otro conjunto de valores. En la ciencia de datos, a menudo necesitamos crear nuevas representaciones a partir de datos existentes, o transformar datos del formato actual al formato que queremos que tengan más adelante. Los mapas son los que se encargan de este trabajo, por lo que son extremadamente importantes para realizarlo.

Hay dos métodos de mapeo que usarás a menudo.

map() es el primero, y ligeramente más simple.

Por ejemplo, imáginate que queremos normalizar el valor del campo `Annual Income` en función de la media, de este modo los salarios que sean negativos serán los que estén por debajo de la media y viceversa.

In [ ]:
media_salario = compras['Annual Income'].mean()
compras["Annual Income"].map(lambda salario: salario - media_salario )

La función que se le pasa a map() debe esperar un único valor de la Serie (un valor de salario, en el ejemplo anterior), y devolver una versión transformada de ese valor. map() devuelve una nueva Serie en la que todos los valores han sido transformados por su función.

apply() es el método equivalente si queremos transformar todo un DataFrame llamando a un método personalizado en cada fila.

In [ ]:
def normalizar_puntuacion_compra(row):
    row['Spending Score'] = row['Spending Score'] // 10
    return row

compras.apply(normalizar_puntuacion_compra, axis='columns')


Si hubierámos llamado a compras.apply() con axis='index', entonces en lugar de pasar una función para transformar cada fila, tendríamos que dar una función para transformar cada columna.

Date cuenta que map() y apply() devuelven Series y DataFrames nuevos y transformados, respectivamente. No modifican los datos originales.

In [ ]:
def suma_valores(column):
    if column.dtype == "int64":
        column = column - 100
    if column.dtype == "object":
        column = column.str.upper()
    return column 

compras.apply(suma_valores, axis='index')

### **3. Uso de función groupby**

Los mapas nos permiten transformar los datos en un DataFrame o Serie un valor a la vez para una columna entera. Sin embargo, a menudo queremos agrupar nuestros datos, y luego hacer algo específico para el grupo en el que se encuentran los datos.

Para esto tenemos la operación groupby().

In [ ]:
compras.groupby("Gender").Gender.count()

La función groupby() internamente crea un grupo de compras que comparten el mismo valor para la columna `gender`. A continuación, para cada uno de estos grupos, tomamos la columna gender() y contamos cuántas veces aparecía. value_counts() es sólo un atajo para esta operación groupby().

Podemos utilizar cualquiera de las funciones de resumen que hemos utilizado antes con estos datos. Por ejemplo, para obtener el salario mínimo que cobran las mujeres y hombres podemos hacer lo siguiente:

In [ ]:
compras.groupby("Gender")["Annual Income"].min()
compras.groupby("Gender")["Annual Income"].max()

Se puede pensar en cada grupo que generamos como una porción de nuestro DataFrame que contiene sólo datos con valores que coinciden. Podemos acceder directamente a este DataFrame utilizando el método apply(), y podemos manipular los datos de la forma que creamos conveniente. Por ejemplo, si queremos obtener el primer salario del primer hombre y de la primera mujer podemos hacer lo siguiente.

In [ ]:
compras.groupby("Gender").apply(lambda df : df["Annual Income"].iloc[0])

Para un control aún más preciso, también se puede agrupar por más de una columna. Por ejemplo, así es como seleccionaríamos los salarios mínimos y máximos agrupados por género y edad:

In [ ]:
compras.groupby(["Age", "Gender"])["Annual Income"].min().head(50)
compras.groupby(["Age", "Gender"])["Annual Income"].min().head(50)

Otro método groupby() digno de mención es agg(), que te permite ejecutar un montón de funciones diferentes en el DataFrame simultáneamente. Por ejemplo, podemos generar un simple resumen estadístico del conjunto de datos de la siguiente manera:

In [ ]:
compras.groupby(['Gender'])['Annual Income'].agg([len, min, max])

### **4. Uso de multi-índices**

En todos los ejemplos que hemos visto hasta ahora hemos estado trabajando con objetos DataFrame o Series con un índice de una sola etiqueta. groupby() es ligeramente diferente en el hecho de que, dependiendo de la operación que ejecutemos, a veces dará como resultado lo que se denomina un multi-índice.

Un multi-índice difiere de un índice normal en que tiene varios niveles. Por ejemplo:

In [ ]:
salario_por_edad_y_genero = compras.groupby(["Age", "Gender"])["Annual Income"].min().head(50)
salario_por_edad_y_genero

index = salario_por_edad_y_genero.index

type(index)

Los índices múltiples disponen de varios métodos para tratar su estructura escalonada que no existen en los índices de un solo nivel. También requieren dos niveles de etiquetas para recuperar un valor. Es un poco complicado al principio tratar con estas estructuras.

Los casos de uso de un multi-índice se detallan en la documentación de Pandas: https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html.

Sin embargo, en general, el método multi-índice que se utiliza más a menudo es el que permite volver a un índice normal, el método reset_index():

In [ ]:
salario_por_edad_y_genero.reset_index()

### **5. Uso de función sort**

Observando de nuevo el dataframe de `salario_por_edad_y_genero` podemos ver que la agrupación devuelve los datos en orden de índice, no en orden de valor. Es decir, al mostrar el resultado de un groupby, el orden de las filas depende de los valores del índice, no de los datos.

El método sort_values() es útil para esto. sort_values() utiliza por defecto una ordenación ascendente, en la que los valores más bajos van primero. Sin embargo, la mayoría de las veces queremos una ordenación descendente, donde los números más altos van primero, con el parámetro `ascending=False` lo conseguimos.

Por ejemplo, para obtener los datos de media de salario agrupados por edad y género podemos hacer lo siguiente.

In [ ]:
media_salario_agg = compras.groupby(['Age', 'Gender'])["Annual Income"].mean().reset_index()

media_salario_agg.sort_values(by="Annual Income", ascending=False)

Para ordenar por valores de índice, utilizamos el método complementario sort_index(). Este método tiene los mismos argumentos y orden por defecto:

In [ ]:
media_salario_agg.sort_index()

Por último, también podemos ordenar por más de una columna a la vez:

In [ ]:
media_salario_agg.sort_values(by=["Age", "Annual Income"], ascending=False)